# Детоксификация текстов на русском языке с помощью модели RuGPT3

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [2]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():

    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [3]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 5.4 MB/s eta 0:00:00


In [4]:
!pip install transformers
!pip install datasets==2.2.2
!pip install razdel
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 40.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.8/346.8 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.9/86.9 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.1/133.1 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.

In [5]:
import os
import pandas as pd
import numpy as np
from argparse import ArgumentParser
from functools import partial
from shutil import rmtree
from tqdm import trange

import numpy as np
from datasets import load_metric, load_dataset
from razdel import tokenize
from transformers import (
    DataCollatorForLanguageModeling,
    TrainingArguments,
    Trainer,
    AutoModelForCausalLM,
    AutoTokenizer,
    AutoModelForSequenceClassification,
    AutoModel
)

from datasets import Dataset, DatasetDict

In [6]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

N_SEEDS = 10
N_EPOCHS = 5
#LR_VALUES = (3e-5, 5e-5)
LR_VALUE = 5e-5
#DECAY_VALUES = (1e-4, 1e-2, 0.1)
#BATCH_SIZES = (32, 64)
BATCH_SIZE = 16

## Загрузка и подготовка данных

In [7]:
DATA_DIR = '/content/'
TRAIN_FILE = DATA_DIR + "train_preprocessed.tsv"
DEV_FILE = DATA_DIR + "dev.tsv"
TEST_FILE = DATA_DIR + "test.tsv"


train_df, dev_df, test_df = map(partial(pd.read_csv, sep='\t'), (TRAIN_FILE, DEV_FILE, TEST_FILE))


train_df['combined'] = '<toxic>' + train_df['toxic_comment'] + '</toxic>' + ' >>>>> ' + '<neutral>' + train_df['neutral_comment1'] + '</neutral>'
with open('train_combined.txt', 'w', encoding='utf-8') as f:
  for line in list(train_df['combined']):
    f.write(line + '\n')

dev_df['combined'] = '<toxic>' + dev_df['toxic_comment'] + '</toxic>' + ' >>>>> ' + '<neutral>' + dev_df['neutral_comment1'] + '</neutral>'
with open('dev_combined.txt', 'w', encoding='utf-8') as f:
  for line in dev_df['combined']:
    f.write(line + '\n')

In [8]:
dataset = load_dataset("text", data_files={"train": "/content/train_combined.txt", "dev": "/content/dev_combined.txt", "test": TEST_FILE})

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-f6d0f22135c4c926/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

## Токенизация

In [9]:
MODEL_NAME = 'ai-forever/rugpt3small_based_on_gpt2'

In [10]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [11]:
def preprocess_examples(examples):
    result = tokenizer(examples['text'], padding=False)
    #result = tokenizer(examples, truncation=True, max_length=256,)
    return result

In [12]:
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

tokenized_dataset = dataset.map(
    preprocess_examples,
    batched=True,
    remove_columns=["text"],
    keep_in_memory=True,
)


  0%|          | 0/7 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

## Training loop

In [13]:
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, pad_token_id=tokenizer.eos_token_id)

training_args = TrainingArguments(
            output_dir=f"checkpoints/",
            overwrite_output_dir=True,
            evaluation_strategy="epoch",
            per_device_train_batch_size=BATCH_SIZE,
            per_device_eval_batch_size=BATCH_SIZE,
            learning_rate=LR_VALUE,
            num_train_epochs=N_EPOCHS,
            lr_scheduler_type="constant",
            optim="adamw_torch",
            save_strategy="epoch",
            save_total_limit=1,
            seed=N_SEEDS,
            dataloader_num_workers=4,
            group_by_length=True,
            report_to="none",
            load_best_model_at_end=True,
            fp16=True,
            #use_cpu=True,
            #bf16=True
            )
trainer = Trainer(
                model=model,
                args=training_args,
                train_dataset=tokenized_dataset["train"],
                eval_dataset=tokenized_dataset["dev"],
                tokenizer=tokenizer,
                data_collator=data_collator,
                )

train_result = trainer.train()
print("train", train_result.metrics)


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast 

Epoch,Training Loss,Validation Loss
1,No log,1.967760
2,1.795000,2.031917
3,1.403700,2.138603
4,1.120700,2.258794
5,0.910900,2.420381


You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This Da

train {'train_runtime': 406.0292, 'train_samples_per_second': 81.657, 'train_steps_per_second': 5.11, 'total_flos': 1000530569088000.0, 'train_loss': 1.2904559785773955, 'epoch': 5.0}


In [14]:

trainer.save_model('/content/gdrive/My Drive/model_ruGPT3_5epochs_preprocessing')

## Инференс


In [15]:
def paraphrase_sentence(text, model, n=None, max_length='auto', temperature=0.0, beams=3):
    text = '<toxic>'+text+'</toxic> >>>>> <neutral>'
    inputs = tokenizer(text, return_tensors='pt', padding=True)['input_ids'].to(model.device)
    if max_length == 'auto':
        max_length = int(inputs.shape[1] * 1.2) + 10
    result = model.generate(
        inputs,
        num_return_sequences=n or 1,
        do_sample=False,
        temperature=temperature,
        repetition_penalty=5.0,
        max_length=max_length,
        bad_words_ids=[[2]],  # unk
        num_beams=beams,
    )
    text = tokenizer.decode(result[0], skip_special_tokens=True)
    text = text.split('<neutral>')[1].split('</ne')[0]
    return text

In [16]:
def paraphrase_batch(texts, model, n=None, max_length='auto', temperature=0.0, beams=3):
    result = []
    for text in texts:
        result.append(paraphrase_sentence(text, model=model, n=n, max_length=max_length, temperature=temperature, beams=3))
    return result

In [17]:
dev = pd.read_csv('/content/dev.tsv', sep='\t')
toxic_inputs = dev['toxic_comment']

In [18]:
from tqdm.auto import tqdm, trange

In [19]:
para_results = []
problematic_batch = [] #if something goes wrong you can track such bathces
batch_size = 8

for i in tqdm(range(0, len(toxic_inputs), batch_size)):
    batch = [sentence for sentence in toxic_inputs[i:i + batch_size]]
    try:
        para_results.extend(paraphrase_batch(batch, model, temperature=0.0))
    except Exception as e:
        print(i)
        para_results.append(toxic_inputs[i:i + batch_size])

  0%|          | 0/100 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


In [20]:
with open('/content/rutgpt3_dev_preprocessing.txt', 'w', encoding='utf-8') as file:
    file.writelines([sentence+'\n' for sentence in para_results])

In [21]:
preds = para_results

In [22]:
import gc
def cleanup():
    gc.collect()
    torch.cuda.empty_cache()

cleanup()

## Оценка
На базе https://github.com/s-nlp/russe_detox_2022/blob/main/evaluation/ru_detoxification_evaluation.ipynb

In [23]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel

In [24]:
def load_model(model_name=None, model=None, tokenizer=None,
               model_class=AutoModelForSequenceClassification, use_cuda=True):
    if model is None:
        if model_name is None:
            raise ValueError('Either model or model_name should be provided')
        model = model_class.from_pretrained(model_name)
        if torch.cuda.is_available() and use_cuda:
            model.cuda()
    if tokenizer is None:
        if model_name is None:
            raise ValueError('Either tokenizer or model_name should be provided')
        tokenizer = AutoTokenizer.from_pretrained(model_name)
    return model, tokenizer

### Style Transfer Accuracy (STA)

In [25]:
def prepare_target_label(model, target_label):
    if target_label in model.config.id2label:
        pass
    elif target_label in model.config.label2id:
        target_label = model.config.label2id.get(target_label)
    elif target_label.isnumeric() and int(target_label) in model.config.id2label:
        target_label = int(target_label)
    else:
        raise ValueError(f'target_label "{target_label}" is not in model labels or ids: {model.config.id2label}.')
    return target_label

In [26]:
def classify_texts(model, tokenizer, texts, second_texts=None, target_label=None, batch_size=32, verbose=False):
    target_label = prepare_target_label(model, target_label)
    res = []
    if verbose:
        tq = trange
    else:
        tq = range
    for i in tq(0, len(texts), batch_size):
        inputs = [texts[i:i+batch_size]]
        if second_texts is not None:
            inputs.append(second_texts[i:i+batch_size])
        inputs = tokenizer(*inputs, return_tensors='pt', padding=True, truncation=True, max_length=512).to(model.device)
        with torch.no_grad():
            preds = torch.softmax(model(**inputs).logits, -1)[:, target_label].cpu().numpy()
        res.append(preds)
    return np.concatenate(res)

In [27]:
def rotation_calibration(data, coef=1.0, px=1, py=1, minimum=0, maximum=1):
    result = (data - px) * coef + py
    if minimum is not None:
        result = np.maximum(minimum, result)
    if maximum is not None:
        result = np.minimum(maximum, result)
    return result

In [28]:
def evaluate_style(
    model,
    tokenizer,
    texts,
    target_label=1,  # 1 is toxic, 0 is neutral
    batch_size=32,
    verbose=False
):
    target_label = prepare_target_label(model, target_label)
    scores = classify_texts(
        model,
        tokenizer,
        texts,
        batch_size=batch_size, verbose=verbose, target_label=target_label
    )
    return rotation_calibration(scores, 0.90)

In [29]:
style_model, style_tokenizer = load_model('SkolkovoInstitute/russian_toxicity_classifier')

In [30]:
accuracy = evaluate_style(
    model = style_model,
    tokenizer = style_tokenizer,
    texts = preds,
    target_label=0,  # 1 is toxic, 0 is neutral
    batch_size=32,
    verbose=True
)

  0%|          | 0/25 [00:00<?, ?it/s]

In [31]:
print(f'Style transfer accuracy (STA):  {np.mean(accuracy)}')

Style transfer accuracy (STA):  0.66190505027771


### Meaning Preservation Score (SIM)

In [32]:
def encode_cls(texts, model, tokenizer, batch_size=32, verbose=False):
    results = []
    if verbose:
        tq = trange
    else:
        tq = range
    for i in tq(0, len(texts), batch_size):
        batch = texts[i: i + batch_size]
        with torch.no_grad():
            out = model(**tokenizer(batch, return_tensors='pt', padding=True, truncation=True).to(model.device))
            embeddings = out.pooler_output
            embeddings = torch.nn.functional.normalize(embeddings).cpu().numpy()
            results.append(embeddings)
    return np.concatenate(results)

In [33]:
def evaluate_cosine_similarity(
    model,
    tokenizer,
    original_texts,
    rewritten_texts,
    batch_size=32,
    verbose=False,
):
    scores = (
        encode_cls(original_texts, model=model, tokenizer=tokenizer, batch_size=batch_size, verbose=verbose)
        * encode_cls(rewritten_texts, model=model, tokenizer=tokenizer, batch_size=batch_size, verbose=verbose)
    ).sum(1)
    return rotation_calibration(scores, 1.50)

In [34]:
meaning_model, meaning_tokenizer = load_model('cointegrated/LaBSE-en-ru', model_class=AutoModel)

In [35]:
similarity = evaluate_cosine_similarity(
    model = meaning_model,
    tokenizer = meaning_tokenizer,
    original_texts = list(toxic_inputs),
    rewritten_texts = preds,
    batch_size=32,
    verbose=True,
    )

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

In [36]:
print(f'Meaning preservation (SIM):  {np.mean(similarity)}')

Meaning preservation (SIM):  0.7197079658508301


### Fluency score (FL)

In [37]:
def evaluate_cola_relative(
    model,
    tokenizer,
    original_texts,
    rewritten_texts,
    target_label=1,
    batch_size=32,
    verbose=False,
    maximum=0,
):
    target_label = prepare_target_label(model, target_label)
    original_scores = classify_texts(
        model, tokenizer,
        original_texts,
        batch_size=batch_size, verbose=verbose, target_label=target_label
    )
    rewritten_scores = classify_texts(
        model, tokenizer,
        rewritten_texts,
        batch_size=batch_size, verbose=verbose, target_label=target_label
    )
    scores = rewritten_scores - original_scores
    if maximum is not None:
        scores = np.minimum(0, scores)
    return rotation_calibration(scores, 1.15, px=0)

In [38]:
cola_model, cola_tolenizer = load_model('SkolkovoInstitute/rubert-base-corruption-detector')

In [39]:
fluency = evaluate_cola_relative(
    model = cola_model,
    tokenizer = cola_tolenizer,
    original_texts = list(toxic_inputs),
    rewritten_texts = preds,
    target_label=1,
    batch_size=32,
    verbose=True
)

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

In [40]:
print(f'Fluency score (FL):  {np.mean(fluency)}')

Fluency score (FL):  0.7626316547393799


### Joint score (J)

In [41]:
joint = accuracy * similarity * fluency

In [42]:
print(f'Joint score (J):   {np.mean(joint)}')

Joint score (J):   0.3656918704509735


### ChrF1 with references

In [43]:
from nltk.translate.chrf_score import corpus_chrf

In [44]:
df = dev.fillna('')
neutral_references = []
for index, row in df.iterrows():
    neutral_references.append([row['neutral_comment1'], row['neutral_comment2'], row['neutral_comment3']])

In [45]:
corpus_chrf(neutral_references, preds)

0.4930422946460803